In [1]:
import torch
import os

from torchvision import datasets

import numpy as np
from glob import glob

from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import gc

In [2]:
filePath = 'train_images_100'

In [3]:
#Preprocessing
preprocess = {
    filePath : transforms.Compose([
        transforms.CenterCrop(224), #Tensor size
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
    ]),
    
    'test_images_100': transforms.Compose([
        transforms.Resize(256), #Resize to image size
        transforms.CenterCrop(224), #Tensor size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
        
    ])
    
}

image_path = "./"

#Load the images
images = {i: datasets.ImageFolder(os.path.join(image_path, i), preprocess[i])
          for i in [filePath, 'test_images_100']}
loader = {x: torch.utils.data.DataLoader(images[x], batch_size = 20, shuffle = True, num_workers = 0) 
                for x in [filePath, 'test_images_100']}

In [4]:
#Making the CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        self.batch1 = nn.BatchNorm2d(224,3)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        self.batch2 = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        self.batch3 = nn.BatchNorm2d(32)
        
        self.conv4 = nn.Conv2d(64, 128, 3, padding = 1) 
        self.batch4 = nn.BatchNorm2d(64)
        
        self.conv5 = nn.Conv2d(128, 256, 3, padding = 1)
        self.batch5 = nn.BatchNorm2d(128)
        
        self.batch6 = nn.BatchNorm2d(256)
        
        # Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Dropout Layer
        self.drop = nn.Dropout(0.25)
        
        len(images[filePath].classes)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(256*7*7, 512)
        self.fc2 = nn.Linear(512, len(images[filePath].classes))
        
    #Forward function
    def forward(self, x):
        # Relu Conv Layer 1
        x = self.pool(F.relu(self.conv1(x)))
        
        # Relu Conv Layer 2
        x = self.batch2(x)
        x = self.pool(F.relu(self.conv2(x)))
        
        # Relu Conv Layer 3
        x = self.batch3(x)
        x = self.pool(F.relu(self.conv3(x)))
        
        # Relu Conv Layer 4
        x = self.batch4(x)
        x = self.pool(F.relu(self.conv4(x)))
        
        # Relu Conv Layer 5
        x = self.batch5(x)
        x = self.pool(F.relu(self.conv5(x)))
        
        x = self.batch6(x)
        
        # Apply view function
        x = x.view(-1, 256 * 7 * 7)
        
        #FC Forwarding
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        
        # Return output
        return x

In [5]:
#Loss Function
model = CNN()

if (torch.cuda.is_available()):
    torch.cuda.empty_cache()
    
    model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
num_epochs = 100

In [7]:
#Training the model        
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    
    model.train()
    for i, (images, labels) in enumerate (loader[filePath]):
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        
        optimizer.zero_grad()    
        
        logits = model(images)
        
        loss = criterion(logits, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.detach().item()
        
    model.eval()
        
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
        %(epoch, train_loss / i, train_acc/i))
        
test_acc = 0.0
correct = 0
total = 0
for i, (images, labels) in enumerate (loader['test_images_100']):
    if torch.cuda.is_available():
        images, labels = images.cuda(), labels.cuda()
    
    outputs = model(images)
    loss = criterion(outputs, labels)
    pred = torch.max(outputs, 1)[1]
    
    correct = correct + np.sum(np.squeeze(pred.eq(labels.data.view_as(pred))).cpu().numpy())
    total = total + images.size(0)
    
print ("Test Accuracy: %2d%% (%2d/%2d)" % (100.0 * correct / total, correct, total))
        

Epoch: 0 | Loss: 3.6604 | Train Accuracy: 0.00
Epoch: 1 | Loss: 3.1317 | Train Accuracy: 0.00
Epoch: 2 | Loss: 2.4906 | Train Accuracy: 0.00
Epoch: 3 | Loss: 1.8967 | Train Accuracy: 0.00
Epoch: 4 | Loss: 1.4095 | Train Accuracy: 0.00
Epoch: 5 | Loss: 1.0717 | Train Accuracy: 0.00
Epoch: 6 | Loss: 0.7686 | Train Accuracy: 0.00
Epoch: 7 | Loss: 0.6424 | Train Accuracy: 0.00
Epoch: 8 | Loss: 0.5054 | Train Accuracy: 0.00
Epoch: 9 | Loss: 0.4293 | Train Accuracy: 0.00
Epoch: 10 | Loss: 0.3681 | Train Accuracy: 0.00
Epoch: 11 | Loss: 0.2756 | Train Accuracy: 0.00
Epoch: 12 | Loss: 0.2542 | Train Accuracy: 0.00
Epoch: 13 | Loss: 0.2006 | Train Accuracy: 0.00
Epoch: 14 | Loss: 0.1845 | Train Accuracy: 0.00
Epoch: 15 | Loss: 0.1941 | Train Accuracy: 0.00
Epoch: 16 | Loss: 0.1525 | Train Accuracy: 0.00
Epoch: 17 | Loss: 0.1383 | Train Accuracy: 0.00
Epoch: 18 | Loss: 0.1387 | Train Accuracy: 0.00
Epoch: 19 | Loss: 0.1305 | Train Accuracy: 0.00
Epoch: 20 | Loss: 0.1271 | Train Accuracy: 0.00
Ep